COGS 118B Final Project - 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import dataset
app_df = pd.read_csv('datasets/application_record.csv')
credit_df = pd.read_csv('datasets/credit_record.csv')

In [3]:
# Data cleaning
credit_df['STATUS'] = credit_df['STATUS'].replace('X', '-2')
credit_df['STATUS'] = credit_df['STATUS'].replace('C', '-1')
credit_df['STATUS'] = credit_df['STATUS'].astype(int)
credit_df['MONTHS_BALANCE'] = credit_df['MONTHS_BALANCE'].astype(int)

In [21]:
# generate a credit level label for each client based on credit history record
def two_classes(client):
    if client['STATUS'].max() <= 0:
        recent = client[client['MONTHS_BALANCE'] >= -6]
        if recent['STATUS'].max() <= 0:
            return 'Prime'
    return 'Subprime'

label_df_2 = credit_df.groupby('ID').apply(two_classes).to_frame()
label_df_2 = label_df_2.rename(columns={0: 'CLASS'})


def four_classes(client):
    recent = client[client['MONTHS_BALANCE'] >= -6]
    worst_status = client['STATUS'].max()
    worst_recent_status = recent['STATUS'].max()

    if worst_status < 0:
        if worst_recent_status < 0:
            return 'Super-prime'
    if worst_status <= 0:
        if worst_recent_status <= 0:
            return 'Prime'
    if worst_status <= 3:
        if worst_recent_status <= 1:
            return 'Near-prime'
    '''
    if worst_status <= 5:
        if worst_recent_status <= 3:
            return 'Subprime'
    '''
    return 'Subprime'
    
label_df_4 = credit_df.groupby('ID').apply(four_classes).to_frame()
label_df_4 = label_df_4.rename(columns={0: 'CLASS'})

/var/folders/sb/cdw7gr054nvc469rwn15_4q00000gn/T/ipykernel_36378/1480022293.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_df_2 = credit_df.groupby('ID').apply(two_classes).to_frame()
/var/folders/sb/cdw7gr054nvc469rwn15_4q00000gn/T/ipykernel_36378/1480022293.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_df_4 = credit_df.groupby('ID').apply(four_classes).to_frame()


In [22]:
# concat label to the application dataframe
label_app_df_2 = pd.merge(app_df, label_df_2, left_on='ID', right_index=True, how='inner')
label_app_df_4 = pd.merge(app_df, label_df_4, left_on='ID', right_index=True, how='inner')
label_app_df_4.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,CLASS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,Near-prime
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,Near-prime
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,Prime
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Prime
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Subprime


In [ ]:
label_app_df_4['CLASS'].value_counts()

CLASS
Prime          20311
Subprime        9381
Super-prime     3706
Near-prime      3059
Name: count, dtype: int64